# Bespoke Example 1

**Problem**:
We have a tab-separated-file where the second column contains text.
we also have a list of words, called "grey list" we want to look for in said text.

The program has to add a new column to the file which will contain:
- `True` if any of the grey list words is in the text of that row
- `False` if none of the grey list words is present in the text of that row

### exploring

In [1]:
grey_list = ["flag", "it", "either"]

In [2]:
filepath = "bespoke_samples/dummyfile.txt"

with open(filepath, encoding="utf8") as f:
    content = f.read()
    
print(content)

﻿henchmen	<s>The Third Reich finally lowers its flag, Hitler and his henchmen are rendered harmless.</s>	View	2020	2349222	IRE	NONE
either	<s>The Spanish electoral system does not favor them either.</s>	The Outlook	2021	3218937	UK	NONE
either	<s>Without knowing how many people we are talking about either.</s>	The Outlook	2021	3999237	UK	NONE
either	<s>And electric trucks equipped for equipment either.</s>	Euromag	2020	7182040	UK	NONE
either	<s>Many of our new customers since confinement either.</s>	The Tribune	2020	7790669	UK	NONE
or not	<s>Is it a big gap or not that much?</s>	Cinema Times	2020	8162203	UK	NONE
or not	<s>Each character has their own vision of what they think should or should not be done.</s>	Cinema Times	2021	8384123	UK	NONE
or not	<s>Many are for having the freedom to wear it or not.</s>	Gazette	2021	8923223	USA	NONE
or not	<s>We could find him a local accent or not.</s>	Euromag	2021	9401415	UK	NONE



### parsing and extracting input

In [3]:
import pandas as pd

df = pd.read_csv(filepath, sep="\t", header=None)
df

,0,1,2,3,4,5,6
0,henchmen,"<s>The Third Reich finally lowers its flag, Hi...",View,2020,2349222,IRE,NONE
1,either,<s>The Spanish electoral system does not favor...,The Outlook,2021,3218937,UK,NONE
2,either,<s>Without knowing how many people we are talk...,The Outlook,2021,3999237,UK,NONE
3,either,<s>And electric trucks equipped for equipment ...,Euromag,2020,7182040,UK,NONE
4,either,<s>Many of our new customers since confinement...,The Tribune,2020,7790669,UK,NONE
5,or not,<s>Is it a big gap or not that much?</s>,Cinema Times,2020,8162203,UK,NONE
6,or not,<s>Each character has their own vision of what...,Cinema Times,2021,8384123,UK,NONE
7,or not,<s>Many are for having the freedom to wear it ...,Gazette,2021,8923223,USA,NONE
8,or not,<s>We could find him a local accent or not.</s>,Euromag,2021,9401415,UK,NONE


In [12]:
list_of_text = df[1].to_list()
list_of_text

['<s>The Third Reich finally lowers its flag, Hitler and his henchmen are rendered harmless.</s>',
 '<s>The Spanish electoral system does not favor them either.</s>',
 '<s>Without knowing how many people we are talking about either.</s>',
 '<s>And electric trucks equipped for equipment either.</s>',
 '<s>Many of our new customers since confinement either.</s>',
 '<s>Is it a big gap or not that much?</s>',
 '<s>Each character has their own vision of what they think should or should not be done.</s>',
 '<s>Many are for having the freedom to wear it or not.</s>',
 '<s>We could find him a local accent or not.</s>']

### processing

In [13]:
import re

def clean_and_tokenize_row_text(row_text):
    clean_text = re.sub('[\W]'," ",row_text[3:-4])
    return clean_text.split()

In [14]:
sample = df.iloc[1,1]
print(sample)
print(clean_and_tokenize_row_text(sample))

<s>The Spanish electoral system does not favor them either.</s>
['The', 'Spanish', 'electoral', 'system', 'does', 'not', 'favor', 'them', 'either']


In [15]:
full_list = []
for row_text in list_of_text:
    row_tokens = clean_and_tokenize_row_text(row_text)
    full_list.extend(row_tokens)

In [17]:
# If we want to see the most common
from collections import Counter

counter = Counter(full_list)
counter.most_common(10)

[('not', 5),
 ('either', 4),
 ('or', 4),
 ('are', 3),
 ('The', 2),
 ('for', 2),
 ('Many', 2),
 ('of', 2),
 ('it', 2),
 ('a', 2)]

In [20]:
found_column_values = []

for row_text in list_of_text:
    found = False
    
    for grey_word in grey_list:
        if grey_word in row_text:
            found = True
            break
            
    found_column_values.append(found)

In [21]:
df.assign(contains_grey_word=found_column_values)

,0,1,2,3,4,5,6,contains_grey_word
0,henchmen,"<s>The Third Reich finally lowers its flag, Hi...",View,2020,2349222,IRE,NONE,True
1,either,<s>The Spanish electoral system does not favor...,The Outlook,2021,3218937,UK,NONE,True
2,either,<s>Without knowing how many people we are talk...,The Outlook,2021,3999237,UK,NONE,True
3,either,<s>And electric trucks equipped for equipment ...,Euromag,2020,7182040,UK,NONE,True
4,either,<s>Many of our new customers since confinement...,The Tribune,2020,7790669,UK,NONE,True
5,or not,<s>Is it a big gap or not that much?</s>,Cinema Times,2020,8162203,UK,NONE,True
6,or not,<s>Each character has their own vision of what...,Cinema Times,2021,8384123,UK,NONE,False
7,or not,<s>Many are for having the freedom to wear it ...,Gazette,2021,8923223,USA,NONE,True
8,or not,<s>We could find him a local accent or not.</s>,Euromag,2021,9401415,UK,NONE,False


### alternative, we also want all found words in a extra column

In [28]:
found_column_values = []
found_words_column_values = []

for row_text in list_of_text:
    found = False
    row_found_words = []
    
    for grey_word in grey_list:
        if grey_word in row_text:
            found = True
            row_found_words.append(grey_word)
            
    found_column_values.append(found)
    combined_found_words = ", ".join(row_found_words)
    found_words_column_values.append(combined_found_words)

In [31]:
result_df = df.assign(contains_grey_word=found_column_values, found_words=found_words_column_values)
result_df

,0,1,2,3,4,5,6,contains_grey_word,found_words
0,henchmen,"<s>The Third Reich finally lowers its flag, Hi...",View,2020,2349222,IRE,NONE,True,"flag, it"
1,either,<s>The Spanish electoral system does not favor...,The Outlook,2021,3218937,UK,NONE,True,"it, either"
2,either,<s>Without knowing how many people we are talk...,The Outlook,2021,3999237,UK,NONE,True,"it, either"
3,either,<s>And electric trucks equipped for equipment ...,Euromag,2020,7182040,UK,NONE,True,"it, either"
4,either,<s>Many of our new customers since confinement...,The Tribune,2020,7790669,UK,NONE,True,"it, either"
5,or not,<s>Is it a big gap or not that much?</s>,Cinema Times,2020,8162203,UK,NONE,True,it
6,or not,<s>Each character has their own vision of what...,Cinema Times,2021,8384123,UK,NONE,False,
7,or not,<s>Many are for having the freedom to wear it ...,Gazette,2021,8923223,USA,NONE,True,it
8,or not,<s>We could find him a local accent or not.</s>,Euromag,2021,9401415,UK,NONE,False,


### save the results to file

In [33]:
output_filepath = "bespoke_samples/greyword_search_result.tsv"
result_df.to_csv(output_filepath, sep="\t", index=False)